In [ ]:
import cv2
import easyocr
from rich.progress import track
from pathlib import Path

input_folder = Path().parent / "images" / "input"
output_folder = Path().parent / "images" / "text_blur"
blur_coefficient = 100 * 2 + 1


def blur_text_easyocr(image_path, output_path, reader: easyocr.Reader):
    image = cv2.imread(image_path)

    results = reader.readtext(image)

    for bbox, text, confidence in results:
        # if confidence > 0.0:  # Confidence threshold
        x_coords = [int(p[0]) for p in bbox]
        y_coords = [int(p[1]) for p in bbox]

        # Get the top-left (min x, min y) and bottom-right (max x, max y)
        top_left = (min(x_coords), min(y_coords))
        bottom_right = (max(x_coords), max(y_coords))

        # Blur the detected region
        roi = image[top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]]
        try:
            blurred = cv2.GaussianBlur(roi, (blur_coefficient, blur_coefficient), 0)
            image[top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]] = (
                blurred
            )
        except Exception as e:
            print(e)

    cv2.imwrite(output_path, image)


reader = easyocr.Reader(["en", "no"], gpu=True)
files = [x for x in input_folder.iterdir()]

output_folder.mkdir(exist_ok=True)

for file in track(files):
    blur_text_easyocr(file, output_folder / file.name, reader)

ValueError: ({'nb'}, 'is not supported')